## Import and Configuration Section

In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import json
import optuna
from functools import partial
from joblib import Memory
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Configuration Class

Configuration class  
Defines key parameters for the model:
- TARGET: The target variable to predict ('AWS' - Accumulated Water Sum)
- USE_LAG_FEATURES: Whether to use lagged values of the target as features
- USE_ROLLING_STATISTICS: Whether to include rolling mean and std as features

In [2]:
# Setup memory cache
cache_dir = './joblib_cache'
memory = Memory(cache_dir, verbose=0)

class Config:
    TARGET = 'AWS'
    USE_LAG_FEATURES = True
    USE_ROLLING_STATISTICS = True

## Data Paths and Feature Selection

Paths and Feature Selection  
- Setting up the base path for the dataset  
- Defining the months and folds for cross-validation  
- Selected features represent atmospheric variables most relevant for prediction  
- DEVICE configuration for CPU/GPU usage

In [3]:
# Paths
base_path = "/kaggle/input/ai-dataimputedataset-k-fold"
months = ["2019-04", "2019-10", "2020-04", "2020-10"]
folds = [f"fold_{i}" for i in range(1, 6)]

# Define selected features
selected_features = [
    'TCW', 'TCLW', 'R250', 'R500', 'R850', 'U850', 'V850', 'EWSS', 'KX', 'CAPE', 'SSHF', 'PEV'
]

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model parameters - now configurable via Optuna
MODEL_CONFIG = {
    "04": {"BATCH_SIZE": 64, "EPOCHS": 10},
    "10": {"BATCH_SIZE": 128, "EPOCHS": 15},
}

## Feature Engineering Functions

### Lag Feature Creation

This cached function creates time-lagged features of the target variable:
- Important for time series forecasting as past values help predict future values  
- Groups by ROW, COL to maintain spatial context  
- Returns test dataframe with added lag features

In [4]:
# Create lag features
@memory.cache
def create_lag_features(train_df, test_df, target_column, lag_steps, groupby_cols):
    """Create lag features for the target column"""
    result_df = test_df.copy()
    
    # Combine train and test for continuous time series
    combined_df = pd.concat([train_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
    
    # Create lag features
    for lag in lag_steps:
        combined_df[f'{target_column}_lag{lag}'] = combined_df.groupby(groupby_cols)[target_column].shift(lag)
    
    # Extract only the test portion with lag features
    result_df = combined_df.iloc[len(train_df):].reset_index(drop=True)

    return result_df

### Rolling Statistics Creation

Creates rolling window statistics (mean and standard deviation):
- Captures trends and volatility in the time series  
- Uses different window sizes to capture short and long-term patterns  
- Groups by ROW, COL to maintain spatial relationships

In [5]:
# Create rolling statistics
@memory.cache
def create_rolling_statistics(train_df, test_df, target_column, window_sizes, groupby_cols):
    """Create rolling statistics features for the target column"""
    result_df = test_df.copy()
    
    # Combine train and test for continuous rolling stats
    combined_df = pd.concat([train_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
    
    # Create rolling features
    for window in window_sizes:
        # Rolling mean
        combined_df[f'{target_column}_rollmean_{window}'] = combined_df.groupby(groupby_cols)[target_column].transform(
            lambda x: x.rolling(window, min_periods=1).mean())
        # Rolling std
        combined_df[f'{target_column}_rollstd_{window}'] = combined_df.groupby(groupby_cols)[target_column].transform(
            lambda x: x.rolling(window, min_periods=1).std())
    
    # Extract only the test portion with rolling features
    result_df = combined_df.iloc[len(train_df):].reset_index(drop=True)

    return result_df

In [6]:
# Handle missing values
@memory.cache
def handle_missing_values(df, lag_steps, window_sizes):
    """Handle missing values in the dataframe"""
    result_df = df.copy()
    
    # Fill NaN values in lag features with 0
    for lag in lag_steps:
        lag_col = f'{Config.TARGET}_lag{lag}'
        if lag_col in result_df.columns:
            result_df[lag_col] = result_df[lag_col].fillna(0)
    
    # Fill NaN values in rolling features with 0
    for window in window_sizes:
        mean_col = f'{Config.TARGET}_rollmean_{window}'
        std_col = f'{Config.TARGET}_rollstd_{window}'
        
        if mean_col in result_df.columns:
            result_df[mean_col] = result_df[mean_col].fillna(0)
        
        if std_col in result_df.columns:
            result_df[std_col] = result_df[std_col].fillna(0)
    
    # Fill remaining NaNs with 0
    result_df = result_df.fillna(0)
    
    return result_df

## Data Loading and Processing

Load and Process Data Function  
Core function for data preparation:
1. Loads data from CSV  
2. Filters to keep only selected features  
3. Handles target variable conversion and NaN/inf values  
4. Creates lag features and rolling statistics when appropriate  
5. Returns cleaned and feature-enhanced dataframe

In [7]:
# Modified load_and_process_data function to filter features
@memory.cache
def load_and_process_data(file_path, train_file_path=None, lag_steps=None, window_sizes=None):
    """
    Load and process data from file_path with optional lag features and rolling statistics
    If train_file_path is provided, use it for creating lag and rolling features
    """
    
    try:
        raw_df = pd.read_csv(file_path)
        
        # Filter to keep only selected features plus essential columns
        essential_cols = ['DATETIME', 'ROW', 'COL', Config.TARGET]
        feature_cols = [col for col in selected_features if col in raw_df.columns]
        filtered_cols = essential_cols + feature_cols
        
        # Keep only needed columns
        raw_df = raw_df[filtered_cols]
        
        # Convert target to numeric and handle NaN/inf values
        raw_df[Config.TARGET] = pd.to_numeric(raw_df[Config.TARGET], errors='coerce')
        raw_df = raw_df.replace([np.inf, -np.inf], np.nan)
        df = raw_df.dropna(subset=[Config.TARGET]).copy()
        
        # Sort by datetime for proper sequence handling
        if 'DATETIME' in df.columns:
            df = df.sort_values("DATETIME").reset_index(drop=True)
        
        # If we're in train mode or not creating lag/rolling features
        if train_file_path is None or (lag_steps is None and window_sizes is None):
            return df
        
        # Otherwise, we're in eval mode and need to carefully create features
        train_df = pd.read_csv(train_file_path)
        
        # Filter training data to keep only selected features
        train_df = train_df[filtered_cols]
        
        train_df[Config.TARGET] = pd.to_numeric(train_df[Config.TARGET], errors='coerce')
        train_df = train_df.replace([np.inf, -np.inf], np.nan)
        train_df = train_df.dropna(subset=[Config.TARGET]).copy()
        
        if 'DATETIME' in train_df.columns:
            train_df = train_df.sort_values("DATETIME").reset_index(drop=True)
        
        # Create lag features if needed
        if Config.USE_LAG_FEATURES and lag_steps:
            df = create_lag_features(train_df, df, Config.TARGET, lag_steps, ['ROW', 'COL'])
        
        # Create rolling statistics if needed
        if Config.USE_ROLLING_STATISTICS and window_sizes:
            df = create_rolling_statistics(train_df, df, Config.TARGET, window_sizes, ['ROW', 'COL'])

        if lag_steps or window_sizes:
            df = handle_missing_values(df, lag_steps or [], window_sizes or [])
        
        return df
        
    except Exception as e:
        print(f"[ERRORS] Error loading or processing data: {str(e)}")
        return pd.DataFrame()

## Model Definition

LSTM Model Architecture  
Neural network model for time series forecasting:
- Uses LSTM (Long Short-Term Memory) layers for sequence modeling  
- Configurable input size, hidden size, and number of layers  
- Dropout for regularization to prevent overfitting  
- Final fully connected layer maps to prediction output  
- Supports multi-step forecasting with configurable time_step_out

In [8]:
# Enhanced LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.0, time_step_out=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, time_step_out)
        self.time_step_out = time_step_out

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

## Sequence Creation

Transforms tabular data into sequences for LSTM training:
- Creates sliding windows of input sequences and target values  
- Maintains spatial grouping by ROW, COL  
- Configurable input length (time_step_in), prediction horizon (time_step_out)  
- Supports stride parameter to control overlap between sequences  
- Returns PyTorch tensors ready for model input

In [9]:
# Create sequences with configurable input and output time steps
def create_sequences(df, input_cols, target_col, time_step_in, time_step_out=1, stride=1):
    """
    Create sequences from dataframe with configurable input and output time steps
    - time_step_in: number of time steps for input
    - time_step_out: number of future steps to predict
    - stride: step size for sliding window
    """
    sequences, targets = [], []
    grouped = df.groupby(['ROW', 'COL'])
    
    for _, group in grouped:
        # Make sure group is sorted by time
        if 'DATETIME' in group.columns:
            group = group.sort_values("DATETIME")
            
        data = group[input_cols].values
        target_data = group[target_col].values
        
        if len(data) < time_step_in + time_step_out:
            continue
        
        for i in range(0, len(data) - time_step_in - time_step_out + 1, stride):
            seq = data[i:i+time_step_in]
            if time_step_out == 1:
                target = target_data[i+time_step_in]
                targets.append(target)
            else:
                target = target_data[i+time_step_in:i+time_step_in+time_step_out]
                targets.append(target)
            sequences.append(seq)
            
    if not sequences:
        return torch.tensor([]), torch.tensor([])
    
    if time_step_out == 1:
        return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).unsqueeze(1)
    else:
        return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)

## Data Loader Preparation

Comprehensive function that:
1. Loads train, validation, and test datasets  
2. Creates lag features and rolling statistics consistently across datasets  
3. Constructs feature columns based on configuration  
4. Creates input sequences and targets for each dataset  
5. Returns PyTorch DataLoaders ready for model training

In [10]:
def prepare_data_loaders(month, fold, lag_steps, window_sizes, time_step_in, time_step_out, batch_size, stride=1):
    """Prepare data loaders with specific time steps and features"""
    folder = os.path.join(base_path, month, fold)
    
    # Load train data
    train_df = load_and_process_data(os.path.join(folder, "processed_train.csv"))
    
    # Load validation data
    val_df = load_and_process_data(os.path.join(folder, "processed_val.csv"))
    
    # Load test data
    test_df = load_and_process_data(os.path.join(folder, "merged_test.csv"))
    
    if train_df.empty or val_df.empty or test_df.empty:
        print("[WARNINGS] One or more datasets are empty")
        return None, None, None, 0
    
    # Sort data by time
    if 'DATETIME' in train_df.columns:
        train_df = train_df.sort_values("DATETIME").reset_index(drop=True)
    if 'DATETIME' in val_df.columns:
        val_df = val_df.sort_values("DATETIME").reset_index(drop=True)
    if 'DATETIME' in test_df.columns:
        test_df = test_df.sort_values("DATETIME").reset_index(drop=True)
    
    # Create features before further processing
    # 1. Create lag features separately for each dataset
    if Config.USE_LAG_FEATURES and lag_steps:
        # Create lag features for train set using itself
        for lag in lag_steps:
            train_df[f'{Config.TARGET}_lag{lag}'] = train_df.groupby(['ROW', 'COL'])[Config.TARGET].shift(lag)
        
        # Create lag features for validation set using train + val
        train_val_df = pd.concat([train_df, val_df]).sort_values('DATETIME').reset_index(drop=True)
        for lag in lag_steps:
            train_val_df[f'{Config.TARGET}_lag{lag}'] = train_val_df.groupby(['ROW', 'COL'])[Config.TARGET].shift(lag)
        val_df = train_val_df.iloc[len(train_df):].reset_index(drop=True)
        
        # Create lag features for test set using train + val + test
        full_df = pd.concat([train_df, val_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
        for lag in lag_steps:
            full_df[f'{Config.TARGET}_lag{lag}'] = full_df.groupby(['ROW', 'COL'])[Config.TARGET].shift(lag)
        test_df = full_df.iloc[len(train_df) + len(val_df):].reset_index(drop=True)
    
    # 2. Create rolling statistics separately for each dataset
    if Config.USE_ROLLING_STATISTICS and window_sizes:
        # Create rolling stats for train set
        for window in window_sizes:
            train_df[f'{Config.TARGET}_rollmean_{window}'] = train_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).mean())
            train_df[f'{Config.TARGET}_rollstd_{window}'] = train_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).std())
        
        # Create rolling stats for validation set
        train_val_df = pd.concat([train_df, val_df]).sort_values('DATETIME').reset_index(drop=True)
        for window in window_sizes:
            train_val_df[f'{Config.TARGET}_rollmean_{window}'] = train_val_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).mean())
            train_val_df[f'{Config.TARGET}_rollstd_{window}'] = train_val_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).std())
        val_df = train_val_df.iloc[len(train_df):].reset_index(drop=True)
        
        # Create rolling stats for test set
        full_df = pd.concat([train_df, val_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
        for window in window_sizes:
            full_df[f'{Config.TARGET}_rollmean_{window}'] = full_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).mean())
            full_df[f'{Config.TARGET}_rollstd_{window}'] = full_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).std())
        test_df = full_df.iloc[len(train_df) + len(val_df):].reset_index(drop=True)
    
    # Handle missing values
    train_df = train_df.fillna(0)
    val_df = val_df.fillna(0)
    test_df = test_df.fillna(0)
    
    basic_cols = [col for col in selected_features if col in train_df.columns]
    lag_cols = [f'{Config.TARGET}_lag{lag}' for lag in lag_steps if f'{Config.TARGET}_lag{lag}' in train_df.columns]
    roll_cols = []
    
    for window in window_sizes:
        mean_col = f'{Config.TARGET}_rollmean_{window}'
        std_col = f'{Config.TARGET}_rollstd_{window}'
        if mean_col in train_df.columns:
            roll_cols.append(mean_col)
        if std_col in train_df.columns:
            roll_cols.append(std_col)
    
    feature_cols = basic_cols + lag_cols + roll_cols
    
    # Check if we have any features
    if not feature_cols:
        print("[WARNINGS] No features detected! Creating default lag feature.")
        # Create at least one default lag feature
        default_lag = 1
        for df in [train_df, val_df, test_df]:
            df[f'{Config.TARGET}_lag{default_lag}'] = df.groupby(['ROW', 'COL'])[Config.TARGET].shift(default_lag)
            df = df.fillna(0)
        
        feature_cols = [f'{Config.TARGET}_lag{default_lag}']
    
    
    # Create sequences
    train_x, train_y = create_sequences(train_df, feature_cols, Config.TARGET, time_step_in, time_step_out, stride)
    val_x, val_y = create_sequences(val_df, feature_cols, Config.TARGET, time_step_in, time_step_out, stride)
    test_x, test_y = create_sequences(test_df, feature_cols, Config.TARGET, time_step_in, time_step_out, stride)
    
    if train_x.numel() == 0 or val_x.numel() == 0 or test_x.numel() == 0:
        print("[WARNINGS] One or more sequence sets are empty")
        return None, None, None, 0
    
    # Create data loaders
    train_loader = DataLoader(TensorDataset(train_x, train_y), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(val_x, val_y), batch_size=batch_size)
    test_loader = DataLoader(TensorDataset(test_x, test_y), batch_size=batch_size)
    
    return train_loader, val_loader, test_loader, train_x.shape[2]

## Model Training

Model Training Function  
Implements the training loop with early stopping:
- Tracks loss on each batch and epoch  
- Implements patience-based early stopping to prevent overfitting  
- Moves data to the appropriate device (CPU/GPU)  
- Returns best loss value for monitoring

In [11]:
def train_model(model, loader, optimizer, loss_fn, epochs, patience=3):
    """Train model with early stopping"""
    model.train()
    best_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        running_loss = 0.0
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * xb.size(0)
        
        avg_loss = running_loss / len(loader.dataset)
        print(f"   Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
        
        # Early stopping check
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"   Early stopping at epoch {epoch+1}")
                break
    
    return best_loss

## Model Evaluation

Model Evaluation  
Calculates performance metrics on validation/test data:
- RMSE (Root Mean Squared Error)  
- Bias  
- R-squared  
- CSI (Critical Success Index)  
- Handles both single-step and multi-step prediction evaluation

In [12]:
def evaluate_model(model, loader):
    """Evaluate model and return metrics"""
    model.eval()
    preds, targets = [], []
    
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            pred = model(xb).cpu()
            preds.append(pred)
            targets.append(yb)
    
    preds = torch.cat(preds).squeeze().numpy()
    targets = torch.cat(targets).squeeze().numpy()
    
    # Handle multi-step output
    if len(preds.shape) > 1 and preds.shape[1] > 1:
        # For multi-step evaluation, we'll calculate metrics on the last predicted step
        preds_last = preds[:, -1]
        targets_last = targets[:, -1] if len(targets.shape) > 1 else targets
        
        rmse = mean_squared_error(targets_last, preds_last, squared=False)
        bias = np.mean(preds_last - targets_last)
        r = r2_score(targets_last, preds_last)
        csi = np.sum((preds_last > 0.1) & (targets_last > 0.1)) / (np.sum((preds_last > 0.1) | (targets_last > 0.1)) + 1e-9)
    else:
        rmse = mean_squared_error(targets, preds, squared=False)
        bias = np.mean(preds - targets)
        r = r2_score(targets, preds)
        csi = np.sum((preds > 0.1) & (targets > 0.1)) / (np.sum((preds > 0.1) | (targets > 0.1)) + 1e-9)
    
    return rmse, bias, r, csi

## Optuna Hyperparameter Optimization

Optuna Objective Function  
Defines the optimization objective for hyperparameter tuning:
- Searches over model architecture parameters (hidden size, layers, dropout)  
- Optimizes learning rate and sequence configurations  
- Tunes feature engineering parameters (lag steps, window sizes)  
- Performs cross-validation across folds for robust parameter selection  
- Returns mean validation RMSE as the objective to minimize

In [13]:
def objective(trial, month):
    """Optuna objective function using cross-validation across folds"""
    # Hyperparameters to tune
    hidden_size = trial.suggest_int("hidden_size", 32, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    time_step_in = trial.suggest_int("time_step_in", 12, 48)
    time_step_out = trial.suggest_int("time_step_out", 1, 6)
    stride = trial.suggest_int("stride", 1, 6)
    
    # Lag steps and window sizes
    num_lags = trial.suggest_int("num_lags", 1, 3)
    lag_steps = [trial.suggest_int(f"lag_{i}", 1, 12) for i in range(num_lags)]
    
    num_windows = trial.suggest_int("num_windows", 1, 3)
    window_sizes = [trial.suggest_int(f"window_{i}", 3, 24) for i in range(num_windows)]
    
    # Get batch size from config
    config_key = month.split("-")[1]
    config = MODEL_CONFIG[config_key]
    batch_size = config["BATCH_SIZE"]
    
    # Cross-validation across folds
    fold_val_rmses = []
    
    for fold in folds:
        try:
            # Prepare data loaders (only train and validation, no test)
            train_loader, val_loader, _, input_size = prepare_data_loaders(
                month, fold, lag_steps, window_sizes, time_step_in, time_step_out, batch_size, stride
            )
            
            if train_loader is None or val_loader is None:
                # Skip this fold if data preparation failed
                continue
            
            # Create model with trial parameters
            model = LSTMModel(
                input_size=input_size, 
                hidden_size=hidden_size, 
                num_layers=num_layers, 
                dropout=dropout,
                time_step_out=time_step_out
            ).to(DEVICE)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            loss_fn = nn.MSELoss()
            
            # Train for a small number of epochs during hyperparameter search
            train_model(model, train_loader, optimizer, loss_fn, epochs=5, patience=2)
            
            # Evaluate on validation set only
            val_rmse, _, _, _ = evaluate_model(model, val_loader)
            fold_val_rmses.append(val_rmse)
            
        except Exception as e:
            print(f"Error in fold {fold}: {str(e)}")
            continue
    
    # Return mean validation RMSE across folds
    if fold_val_rmses:
        mean_val_rmse = sum(fold_val_rmses) / len(fold_val_rmses)
        return mean_val_rmse
    else:
        return float('inf')

## Testing and Final Evaluation

Test Set Evaluation  
Uses best hyperparameters to train final models:
1. Trains model with optimal hyperparameters  
2. Evaluates on validation and test sets  
3. Saves model weights  
4. Returns comprehensive metrics dictionary

In [14]:
def evaluate_on_test(month, fold, best_params):
    """Train and evaluate model with best parameters on test set"""
    print(f"\n====== Training model for {month}, {fold} with best parameters")
    
    # Extract parameters
    hidden_size = best_params["hidden_size"]
    num_layers = best_params["num_layers"]
    dropout = best_params["dropout"]
    lr = best_params["lr"]
    time_step_in = best_params["time_step_in"]
    time_step_out = best_params["time_step_out"]
    stride = best_params["stride"]
    
    # Extract lag steps and window sizes
    num_lags = best_params["num_lags"]
    lag_steps = [best_params[f"lag_{i}"] for i in range(num_lags)]
    
    num_windows = best_params["num_windows"]
    window_sizes = [best_params[f"window_{i}"] for i in range(num_windows)]
    
    # Get config
    config_key = month.split("-")[1]
    config = MODEL_CONFIG[config_key]
    batch_size = config["BATCH_SIZE"]
    epochs = config["EPOCHS"]
    
    # Prepare data
    train_loader, val_loader, test_loader, input_size = prepare_data_loaders(
        month, fold, lag_steps, window_sizes, time_step_in, time_step_out, batch_size, stride
    )
    
    if train_loader is None or val_loader is None or test_loader is None:
        print("[ERRORS] Failed to prepare data")
        return None
    
    # Create model
    model = LSTMModel(
        input_size=input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout,
        time_step_out=time_step_out
    ).to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    
    # Train model on train set
    print(f"====== Training model on train set...")
    train_model(model, train_loader, optimizer, loss_fn, epochs=epochs, patience=5)
    
    # Evaluate on validation set
    print("====== Evaluating on validation set...")
    val_rmse, val_bias, val_r, val_csi = evaluate_model(model, val_loader)
    print(f"  Val RMSE: {val_rmse:.4f}, Bias: {val_bias:.4f}, R²: {val_r:.4f}, CSI: {val_csi:.4f}")
    
    # Evaluate on test set
    print("====== Evaluating on test set...")
    test_rmse, test_bias, test_r, test_csi = evaluate_model(model, test_loader)
    print(f"  Test RMSE: {test_rmse:.4f}, Bias: {test_bias:.4f}, R²: {test_r:.4f}, CSI: {test_csi:.4f}")
    
    # Save model for this fold
    model_path = f"model_{month}_{fold}.pt"
    torch.save(model.state_dict(), model_path)
    print(f" Model saved to {model_path}")
    
    return {
        "month": month,
        "fold": fold,
        "test_rmse": round(test_rmse, 4),
        "test_bias": round(test_bias, 4),
        "test_r": round(test_r, 4),
        "test_csi": round(test_csi, 4),
        "val_rmse": round(val_rmse, 4),
        "val_bias": round(val_bias, 4),
        "val_r": round(val_r, 4),
        "val_csi": round(val_csi, 4),
        "model": model
    }

## Main Execution Function

Optuna Optimization Main Function  
Orchestrates the entire model building process:
1. For each month:
   a. Runs hyperparameter optimization with cross-validation  
   b. Saves best parameters  
   c. Evaluates on test set for each fold  
   d. Saves best model  
2. Calculates and reports performance metrics:
   a. Per-month metrics  
   b. Overall mean performance  
3. Saves results to CSV files for further analysis

In [15]:
def run_optuna_optimization():
    """Run Optuna optimization with cross-validation and final test evaluation"""
    month_results = {}
    
    for month in months:
        print(f"\n###### Processing month: {month}")
        
        # Step 1: Find the best hyperparameters using cross-validation
        study_name = f"{month}_study"
        study = optuna.create_study(direction="minimize", study_name=study_name)
        
        # Create a partial function with fixed month
        objective_func = partial(objective, month=month)
        
        # Run optimization with cross-validation
        try:
            study.optimize(objective_func, n_trials=20, timeout=3600)  # 1 hour timeout
            
            best_params = study.best_params
            print(f" Best parameters for {month}:")
            for key, value in best_params.items():
                print(f"  {key}: {value}")
            
            # Save best parameters for the month
            best_params_path = f"best_params_{month}.json"
            with open(best_params_path, "w") as f:
                json.dump(best_params, f, indent=2)
            
            print(f" Best parameters saved to {best_params_path}")
            
            # Step 2: Final evaluation on test set for each fold
            fold_test_results = []
            for fold in folds:
                print(f"\n Final evaluation on {month}, {fold} test set")
                result = evaluate_on_test(month, fold, best_params)
                if result:
                    fold_test_results.append(result)
            
            # Calculate mean performance across folds
            if fold_test_results:
                mean_rmse = sum(r["test_rmse"] for r in fold_test_results) / len(fold_test_results)
                mean_bias = sum(r["test_bias"] for r in fold_test_results) / len(fold_test_results)
                mean_r = sum(r["test_r"] for r in fold_test_results) / len(fold_test_results)
                mean_csi = sum(r["test_csi"] for r in fold_test_results) / len(fold_test_results)
                
                # Save best fold result (lowest RMSE)
                best_fold_result = min(fold_test_results, key=lambda x: x["test_rmse"])
                best_model = best_fold_result["model"]
                best_model_path = f"best_model_{month}.pt"
                torch.save(best_model.state_dict(), best_model_path)
                
                # Store results for this month
                month_results[month] = {
                    "month": month,
                    "best_fold": best_fold_result["fold"],
                    "best_test_rmse": best_fold_result["test_rmse"],
                    "best_test_bias": best_fold_result["test_bias"],
                    "best_test_r": best_fold_result["test_r"],
                    "best_test_csi": best_fold_result["test_csi"],
                    "mean_test_rmse": round(mean_rmse, 4),
                    "mean_test_bias": round(mean_bias, 4),
                    "mean_test_r": round(mean_r, 4),
                    "mean_test_csi": round(mean_csi, 4),
                    "best_params": best_params,
                    "fold_results": fold_test_results
                }
                
                print(f"\n Mean test performance for {month}:")
                print(f"  RMSE: {mean_rmse:.4f}, Bias: {mean_bias:.4f}, R²: {mean_r:.4f}, CSI: {mean_csi:.4f}")
                print(f" Best fold test performance for {month} (fold {best_fold_result['fold']}):")
                print(f"  RMSE: {best_fold_result['test_rmse']:.4f}, Bias: {best_fold_result['test_bias']:.4f}, R²: {best_fold_result['test_r']:.4f}, CSI: {best_fold_result['test_csi']:.4f}")
                print(f" Best model for {month} saved to {best_model_path}")
            
        except Exception as e:
            print(f"[ERRORS] Error during optimization for {month}: {str(e)}")
            continue
    
    # Calculate overall mean performance across months
    if month_results:
        overall_mean_rmse = sum(r["best_test_rmse"] for r in month_results.values()) / len(month_results)
        overall_mean_bias = sum(r["best_test_bias"] for r in month_results.values()) / len(month_results)
        overall_mean_r = sum(r["best_test_r"] for r in month_results.values()) / len(month_results)
        overall_mean_csi = sum(r["best_test_csi"] for r in month_results.values()) / len(month_results)
        
        print("\n Overall mean performance across all months:")
        print(f"  RMSE: {overall_mean_rmse:.4f}, Bias: {overall_mean_bias:.4f}, R²: {overall_mean_r:.4f}, CSI: {overall_mean_csi:.4f}")
    
    # Create a dataframe with month results
    month_results_df = pd.DataFrame([
        {
            "month": month,
            "best_fold": data["best_fold"],
            "best_test_rmse": data["best_test_rmse"],
            "best_test_bias": data["best_test_bias"],
            "best_test_r": data["best_test_r"],
            "best_test_csi": data["best_test_csi"],
            "mean_test_rmse": data["mean_test_rmse"],
            "mean_test_bias": data["mean_test_bias"],
            "mean_test_r": data["mean_test_r"],
            "mean_test_csi": data["mean_test_csi"],
        } for month, data in month_results.items()
    ])
    
    # Display month-wise results
    print("\n Results by Month:")
    print(month_results_df)
    
    # Save results
    month_results_df.to_csv("lstm_month_results.csv", index=False)
    
    # Save detailed results
    all_fold_results = []
    for month, data in month_results.items():
        for fold_result in data["fold_results"]:
            all_fold_results.append({
                "month": month,
                "fold": fold_result["fold"],
                "test_rmse": fold_result["test_rmse"],
                "test_bias": fold_result["test_bias"],
                "test_r": fold_result["test_r"],
                "test_csi": fold_result["test_csi"],
                "val_rmse": fold_result["val_rmse"],
                "val_bias": fold_result["val_bias"],
                "val_r": fold_result["val_r"],
                "val_csi": fold_result["val_csi"],
            })
    
    all_fold_results_df = pd.DataFrame(all_fold_results)
    all_fold_results_df.to_csv("lstm_all_fold_results.csv", index=False)
    
    print(" Results saved to lstm_month_results.csv and lstm_all_fold_results.csv")
    
    return month_results

In [16]:
# Execute optimization
if __name__ == "__main__":
    run_optuna_optimization()

[I 2025-05-07 02:05:06,176] A new study created in memory with name: 2019-04_study



###### Processing month: 2019-04
   Epoch 1/5, Loss: 0.5852
   Epoch 2/5, Loss: 0.5628
   Epoch 3/5, Loss: 0.5295
   Epoch 4/5, Loss: 0.4946
   Epoch 5/5, Loss: 0.4759
   Epoch 1/5, Loss: 0.3093
   Epoch 2/5, Loss: 0.2883
   Epoch 3/5, Loss: 0.2711
   Epoch 4/5, Loss: 0.2579
   Epoch 5/5, Loss: 0.2459
   Epoch 1/5, Loss: 0.2926
   Epoch 2/5, Loss: 0.2747
   Epoch 3/5, Loss: 0.2611
   Epoch 4/5, Loss: 0.2512
   Epoch 5/5, Loss: 0.2395
   Epoch 1/5, Loss: 0.3414
   Epoch 2/5, Loss: 0.3209
   Epoch 3/5, Loss: 0.3106
   Epoch 4/5, Loss: 0.3009
   Epoch 5/5, Loss: 0.2942
   Epoch 1/5, Loss: 0.2669
   Epoch 2/5, Loss: 0.2514
   Epoch 3/5, Loss: 0.2410
   Epoch 4/5, Loss: 0.2339
   Epoch 5/5, Loss: 0.2292


[I 2025-05-07 02:08:21,243] Trial 0 finished with value: 0.42700807750225067 and parameters: {'hidden_size': 162, 'num_layers': 1, 'dropout': 0.43565981286398253, 'lr': 0.0003321592347988165, 'time_step_in': 16, 'time_step_out': 2, 'stride': 1, 'num_lags': 3, 'lag_0': 10, 'lag_1': 2, 'lag_2': 7, 'num_windows': 3, 'window_0': 21, 'window_1': 14, 'window_2': 22}. Best is trial 0 with value: 0.42700807750225067.


   Epoch 1/5, Loss: 0.6807
   Epoch 2/5, Loss: 0.6696
   Epoch 3/5, Loss: 0.6513
   Epoch 4/5, Loss: 0.6205
   Epoch 5/5, Loss: 0.6089
   Epoch 1/5, Loss: 0.3186
   Epoch 2/5, Loss: 0.3110
   Epoch 3/5, Loss: 0.3018
   Epoch 4/5, Loss: 0.3025
   Epoch 5/5, Loss: 0.2947
   Epoch 1/5, Loss: 0.2488
   Epoch 2/5, Loss: 0.2499
   Epoch 3/5, Loss: 0.2480
   Epoch 4/5, Loss: 0.2458
   Epoch 5/5, Loss: 0.2466
   Epoch 1/5, Loss: 0.3509
   Epoch 2/5, Loss: 0.3467
   Epoch 3/5, Loss: 0.3510
   Epoch 4/5, Loss: 0.3464
   Epoch 5/5, Loss: 0.3422
   Epoch 1/5, Loss: 0.2731
   Epoch 2/5, Loss: 0.2674
   Epoch 3/5, Loss: 0.2654
   Epoch 4/5, Loss: 0.2636
   Epoch 5/5, Loss: 0.2618


[I 2025-05-07 02:10:07,691] Trial 1 finished with value: 0.32690627723932264 and parameters: {'hidden_size': 187, 'num_layers': 2, 'dropout': 0.4508534705344154, 'lr': 0.0051643475523695526, 'time_step_in': 35, 'time_step_out': 6, 'stride': 5, 'num_lags': 2, 'lag_0': 8, 'lag_1': 7, 'num_windows': 3, 'window_0': 24, 'window_1': 7, 'window_2': 3}. Best is trial 1 with value: 0.32690627723932264.


...

   Epoch 1/5, Loss: 2.9530
   Epoch 2/5, Loss: 2.2015
   Epoch 3/5, Loss: 1.9519
   Epoch 4/5, Loss: 1.8157
   Epoch 5/5, Loss: 1.7569
   Epoch 1/5, Loss: 2.2854
   Epoch 2/5, Loss: 1.6468
   Epoch 3/5, Loss: 1.4717
   Epoch 4/5, Loss: 1.4051
   Epoch 5/5, Loss: 1.3835
   Epoch 1/5, Loss: 2.0538
   Epoch 2/5, Loss: 1.5334
   Epoch 3/5, Loss: 1.4433
   Epoch 4/5, Loss: 1.4004
   Epoch 5/5, Loss: 1.3767
   Epoch 1/5, Loss: 1.9882
   Epoch 2/5, Loss: 1.4430
   Epoch 3/5, Loss: 1.3708
   Epoch 4/5, Loss: 1.3391
   Epoch 5/5, Loss: 1.3068
   Epoch 1/5, Loss: 1.8931
   Epoch 2/5, Loss: 1.4611
   Epoch 3/5, Loss: 1.4049
   Epoch 4/5, Loss: 1.3580


[I 2025-05-07 04:50:49,249] Trial 18 finished with value: 1.2259411096572876 and parameters: {'hidden_size': 195, 'num_layers': 1, 'dropout': 0.07307578138697404, 'lr': 0.00016603306916094203, 'time_step_in': 42, 'time_step_out': 1, 'stride': 5, 'num_lags': 1, 'lag_0': 2, 'num_windows': 1, 'window_0': 14}. Best is trial 0 with value: 1.0021047711372375.


   Epoch 5/5, Loss: 1.3047
   Epoch 1/5, Loss: 2.3160
   Epoch 2/5, Loss: 1.5738
   Epoch 3/5, Loss: 1.4592
   Epoch 4/5, Loss: 1.4538
   Epoch 5/5, Loss: 1.4303
   Epoch 1/5, Loss: 1.8362
   Epoch 2/5, Loss: 1.2737
   Epoch 3/5, Loss: 1.2168
   Epoch 4/5, Loss: 1.2048
   Epoch 5/5, Loss: 1.1974
   Epoch 1/5, Loss: 1.4001
   Epoch 2/5, Loss: 1.2439
   Epoch 3/5, Loss: 1.1767
   Epoch 4/5, Loss: 1.1743
   Epoch 5/5, Loss: 1.1479
   Epoch 1/5, Loss: 1.4081
   Epoch 2/5, Loss: 1.2205
   Epoch 3/5, Loss: 1.1977
   Epoch 4/5, Loss: 1.1682
   Epoch 5/5, Loss: 1.1519
   Epoch 1/5, Loss: 1.3565
   Epoch 2/5, Loss: 1.2221
   Epoch 3/5, Loss: 1.1687
   Epoch 4/5, Loss: 1.1472
   Epoch 5/5, Loss: 1.1155


[I 2025-05-07 04:55:33,808] Trial 19 finished with value: 1.233364462852478 and parameters: {'hidden_size': 239, 'num_layers': 2, 'dropout': 0.1702338301459344, 'lr': 0.009966592825219731, 'time_step_in': 35, 'time_step_out': 2, 'stride': 1, 'num_lags': 2, 'lag_0': 6, 'lag_1': 8, 'num_windows': 2, 'window_0': 12, 'window_1': 14}. Best is trial 0 with value: 1.0021047711372375.


 Best parameters for 2020-10:
  hidden_size: 188
  num_layers: 2
  dropout: 0.1990680107401797
  lr: 0.000127233756628581
  time_step_in: 47
  time_step_out: 1
  stride: 1
  num_lags: 2
  lag_0: 9
  lag_1: 3
  num_windows: 2
  window_0: 12
  window_1: 21
 Best parameters saved to best_params_2020-10.json

 Final evaluation on 2020-10, fold_1 test set

====== Training model for 2020-10, fold_1 with best parameters
====== Training model on train set...
   Epoch 1/15, Loss: 2.4192
   Epoch 2/15, Loss: 1.8416
   Epoch 3/15, Loss: 1.6727
   Epoch 4/15, Loss: 1.5761
   Epoch 5/15, Loss: 1.4827
   Epoch 6/15, Loss: 1.4129
   Epoch 7/15, Loss: 1.3334
   Epoch 8/15, Loss: 1.2935
   Epoch 9/15, Loss: 1.2597
   Epoch 10/15, Loss: 1.2072
   Epoch 11/15, Loss: 1.1953
   Epoch 12/15, Loss: 1.1568
   Epoch 13/15, Loss: 1.1304
   Epoch 14/15, Loss: 1.1076
   Epoch 15/15, Loss: 1.0877
====== Evaluating on validation set...
  Val RMSE: 0.9009, Bias: -0.0838, R²: 0.6362, CSI: 0.6032
====== Evaluating on 